In [2]:
from airfoil_generator_functions import *

In [3]:

mesh = PropellerMesh('./nine_inch_tombo_propeller.msh')

mesh loaded successful


In [11]:
normal_vector = [1, 0, 0]
r = 20
R = 110
step = 10
header = ['cross-section position', 'number of upper points', 'number of lower points', 'twist angle', 'chord length', 'l', 't']
with open('jaxa_cross_infor.csv', 'w', encoding='UTF8', newline="") as g:
    # write the header
    writer = csv.writer(g)
    writer.writerow(header)
    for i in range(r, R, step):
        j = round(i/R, 2)
        print(j)
        PropellerCrossSection = [-i, 0, 0]
        slice = mesh.section(plane_origin=[PropellerCrossSection[0], PropellerCrossSection[1], PropellerCrossSection[2]],
                         plane_normal=normal_vector)
        AirfoilSlice2D, to_3D = slice.to_planar()    
        cross_section_points = extract_points_of_slice2D(AirfoilSlice2D)
        convert_points = [cross_section_points[1], -cross_section_points[0]]
        airfoil = []
        for i in range(len(convert_points[0])):
            point = (convert_points[0][i], convert_points[1][i])
            airfoil.append(point)   
        twist_angle, l, t, chord_length = rotate_angle_of_airfoil(airfoil)
        rotated_points = rotate_and_shift(np.array(airfoil), airfoil[l], degrees=-twist_angle) / chord_length
        uppers, lowers = split_upper_and_lower(rotated_points, l, t)
        floatformat = ff.FortranRecordWriter('(1E20.10)')
        intformat = ff.FortranRecordWriter('(I12)')
        
        with open(f'cross_{j}.txt', 'w') as f:
            n_uppers = uppers.shape[0]
            f.write(intformat.write([n_uppers]) + '\n')
            f.write(floatformat.write(uppers[:, 0]) + '\n')
            f.write(floatformat.write(uppers[:, 1]) + '\n')

            n_lowers = lowers.shape[0]
            f.write(intformat.write([n_lowers]) + '\n')
            f.write(floatformat.write(lowers[:, 0]) + '\n')
            f.write(floatformat.write(lowers[:, 1]) + '\n')
            f.close()
        data = [j, n_uppers, n_lowers, twist_angle, chord_length]
        # write the data
        writer.writerow(data)
        plt.figure(figsize = (5, 5))
        PlotSpline(uppers, "blue")
        PlotSpline(lowers, "red")
        plt.axis('equal')
        plt.show()
        plt.close()
    g.close()

PermissionError: [Errno 13] Permission denied: 'jaxa_cross_infor.csv'

In [ ]:
PropellerCrossSection = [-15, 0, 0]
slice = mesh.section(plane_origin=[PropellerCrossSection[0], PropellerCrossSection[1], PropellerCrossSection[2]],
                plane_normal=normal_vector)
AirfoilSlice2D, to_3D = slice.to_planar()    
cross_section_points = extract_points_of_slice2D(AirfoilSlice2D)
convert_points = [cross_section_points[1], -cross_section_points[0]]
airfoil = []
for i in range(len(convert_points[0])):
    point = (convert_points[0][i], convert_points[1][i])
    airfoil.append(point) 
twist_angle, l, t, chord_length = rotate_angle_of_airfoil(airfoil)
rotated_points = rotate_and_shift(np.array(airfoil), airfoil[l], degrees=-twist_angle) / chord_length
uppers, lowers = split_upper_and_lower(rotated_points, l, t)
